## 计算一个专业各个课程的关联度,然后存储起来

In [60]:
import requests
import jutil
import time
import json
import itertools
import math

In [67]:
def get_course_code(speciality_code,grade='2015'):
    '''从数据库查出课程名和代码'''
    sql = "select DISTINCT(course_name),course_code from view_stu_course_mark where speciality_code='%s' and course_type='必'  and grade='grade'" \
        %(speciality_code,grade)
    df = jutil.load_pd_df(sql)
    return df

In [66]:
def cal_relation(code1,code2):
    '''计算相似度调用'''
    url2 = 'http://localhost:8082/course/relation/'
    url = url2+code1+'/'+code2
    re = requests.get(url)
    return re.json()['data']

In [15]:
re = cal_relation('0173186','0571009')
re

{'bg_prob': '0.00', 'prob': '0.00', 'total': 0}

In [7]:
df = get_course_code('0501')
df.head()

,course_code,course_name
0,0173186,工程制图
1,0571009,综合运输概论
2,0573042,物流导论
3,0673054,管理信息系统
4,0673107,网络金融


In [17]:
course_records = df.to_dict('records')
course_records

[{'course_code': '0173186', 'course_name': '工程制图'},
 {'course_code': '0571009', 'course_name': '综合运输概论'},
 {'course_code': '0573042', 'course_name': '物流导论'},
 {'course_code': '0673054', 'course_name': '管理信息系统'},
 {'course_code': '0673107', 'course_name': '网络金融'},
 {'course_code': '1271046', 'course_name': '高等数学BⅠ'},
 {'course_code': '1273022', 'course_name': '高等代数Ⅰ'},
 {'course_code': '1273028', 'course_name': '解析几何'},
 {'course_code': '1371032', 'course_name': '大学物理BI'},
 {'course_code': '1371038', 'course_name': '格致物理'},
 {'course_code': '1572001', 'course_name': '大学计算机基础B'},
 {'course_code': '1671003', 'course_name': '分析化学'},
 {'course_code': '1671065', 'course_name': '土木工程制图II'},
 {'course_code': '1671078', 'course_name': '遗传密码—生命与自然'},
 {'course_code': '1673080', 'course_name': '药用植物认识实习'},
 {'course_code': '2171026', 'course_name': '测量学'},
 {'course_code': '2171084', 'course_name': '高速铁路防火技术'},
 {'course_code': '2572005', 'course_name': '市场营销学'},
 {'course_code': '2671029', 'cour

In [61]:
def drop_nan(num):
    '''如果非数字,则返回0'''
    return 0 if math.isnan(num) else num/100

In [71]:
def get_single_links(course_records):
    '''计算所有课程的两两关系,复杂度为n(n-1)/2'''
    single_links = []
    batch_link = [] # 辅助写入文件的
    begin_time = time.time()
    n = len(course_records)
    for i in range(n-1):
        for j in range(i+1,n):
            code1 = course_records[i]['course_code']
            code2 = course_records[j]['course_code']
            result = cal_relation(code1,code2)
            link = {}
            link['source'] = course_records[i]['course_name']
            link['target'] = course_records[j]['course_name']
            link['prob'] = drop_nan(float(result['prob']))
            link['bg_prob'] = drop_nan(float(result['bg_prob']))
            link['total'] = result['total']
            single_links.append(link)
            batch_link.append(link)
            # 周期性地写入文件,保存结果,因为计算量很大,需要很长时间
            if len(batch_link) > 30 or (i==n-2 and j==n-1):
                with open('single_links.txt','a') as f:
                    for x in batch_link:
                        json.dump(x,f)
                        f.write('\n')
                    end_time = time.time()
                    print('写入%d条信息,耗时%.2f 秒' % (len(batch_link),end_time-begin_time))
                    batch_link.clear()
                    begin_time = end_time
    return single_links

## 测试get_single_links

In [64]:
short_dict = course_records[:10]
single_links = get_single_links(short_dict)
single_links[:5]

写入0条信息,耗时17.04 秒
写入0条信息,耗时7.97 秒


[{'bg_prob': 0.0,
  'prob': 0.0,
  'source': '工程制图',
  'target': '综合运输概论',
  'total': 0},
 {'bg_prob': 0.0, 'prob': 0.0, 'source': '工程制图', 'target': '物流导论', 'total': 0},
 {'bg_prob': 0.0,
  'prob': 0.0,
  'source': '工程制图',
  'target': '管理信息系统',
  'total': 0},
 {'bg_prob': 0.0, 'prob': 0.0, 'source': '工程制图', 'target': '网络金融', 'total': 0},
 {'bg_prob': 0.2667,
  'prob': 0.5581,
  'source': '工程制图',
  'target': '高等数学BⅠ',
  'total': 86}]

## 生成nodes和links数据

In [21]:
def add_one_node_record(nodeRecord,name,prob):
    if name in nodeRecord:
        nodeRecord[name]['edgeCount'] += 1
        nodeRecord[name]['probSum'] += prob
    else :
        record = {}
        record['name'] = name
        record['edgeCount'] = 1
        record['probSum'] = prob
        nodeRecord[name] = record

In [56]:
def get_nodes_and_links(single_links):
    '''根据单个的课程间概率生成echarts展示的数据结构
    包括nodes和links
    nodes单个的结构如下:
    {
        category: 0,
        name: '高等数学',
        value: 10,
        symbolSize: 50 //圆圈的大小,一般大于30,这里我用与之相连的课程数和他们的概率来计算
    }
    links的单个结构如下:
    { source: '高等数学', target: '大学物理' }
    '''
    nodes = []
    nodeRecord = {} # 帮助计算node,存储的结构{name:{edgeCount:0,probSum:0.0}}
    links = []
    for slink in single_links:
        # 舍弃掉那些人数少的例子
        if slink['total'] > 30:
            link = {}
            link['source'] = slink['source']
            link['target'] = slink['target']
            links.append(link)
            add_one_node_record(nodeRecord,slink['source'],slink['prob'])
            add_one_node_record(nodeRecord,slink['target'],slink['prob'])
    # 计算nodes
#     print(nodeRecord)
    for name,v in nodeRecord.items():
        node = {}
        node['category'] = 0
        node['name'] = name
        node['symbolSize'] = v['edgeCount']*3 + v['probSum']*5
        nodes.append(node)
    return links,nodes
    

## 测试生成的nodes和links数据

In [42]:
single_links = []
with open('links.txt','r') as f:
    l = f.readlines()
    for x in l:
        d = json.loads(x)
        single_links.append(d)

In [51]:
links,nodes = get_nodes_and_links(single_links)

{'工程制图': {'name': '工程制图', 'edgeCount': 8, 'probSum': 499.69000000000005}, '高等数学BⅠ': {'name': '高等数学BⅠ', 'edgeCount': 3, 'probSum': 130.91}, '大学物理BI': {'name': '大学物理BI', 'edgeCount': 2, 'probSum': 145.82999999999998}, '测量学': {'name': '测量学', 'edgeCount': 1, 'probSum': 42.34}, '线性代数B': {'name': '线性代数B', 'edgeCount': 2, 'probSum': 106.80000000000001}, '军事技能训练': {'name': '军事技能训练', 'edgeCount': 2, 'probSum': 169.79}, '英语Ⅰ': {'name': '英语Ⅰ', 'edgeCount': 2, 'probSum': 147.97}, '高等数学BⅡ': {'name': '高等数学BⅡ', 'edgeCount': 2, 'probSum': 134.72}, '普通测量实验': {'name': '普通测量实验', 'edgeCount': 1, 'probSum': 69.23}, '综合运输概论': {'name': '综合运输概论', 'edgeCount': 13, 'probSum': nan}, '物流导论': {'name': '物流导论', 'edgeCount': 2, 'probSum': 103.49000000000001}, '分析化学': {'name': '分析化学', 'edgeCount': 1, 'probSum': 58.54}, '物流学': {'name': '物流学', 'edgeCount': 1, 'probSum': 74.7}, '英语综合能力MⅠ': {'name': '英语综合能力MⅠ', 'edgeCount': 1, 'probSum': 93.55}, '计算机程序设计基础A': {'name': '计算机程序设计基础A', 'edgeCount': 1, 'probSum': 68.97}, '无机化学

In [44]:
links

[{'source': '工程制图', 'target': '高等数学BⅠ'},
 {'source': '工程制图', 'target': '大学物理BI'},
 {'source': '工程制图', 'target': '测量学'},
 {'source': '工程制图', 'target': '线性代数B'},
 {'source': '工程制图', 'target': '军事技能训练'},
 {'source': '工程制图', 'target': '英语Ⅰ'},
 {'source': '工程制图', 'target': '高等数学BⅡ'},
 {'source': '工程制图', 'target': '普通测量实验'},
 {'source': '综合运输概论', 'target': '物流导论'},
 {'source': '综合运输概论', 'target': '高等数学BⅠ'},
 {'source': '综合运输概论', 'target': '大学物理BI'},
 {'source': '综合运输概论', 'target': '分析化学'},
 {'source': '综合运输概论', 'target': '线性代数B'},
 {'source': '综合运输概论', 'target': '军事技能训练'},
 {'source': '综合运输概论', 'target': '物流学'},
 {'source': '综合运输概论', 'target': '英语综合能力MⅠ'},
 {'source': '综合运输概论', 'target': '英语Ⅰ'},
 {'source': '综合运输概论', 'target': '高等数学BⅡ'},
 {'source': '综合运输概论', 'target': '计算机程序设计基础A'},
 {'source': '综合运输概论', 'target': '无机化学实验'},
 {'source': '综合运输概论', 'target': '管理学原理'},
 {'source': '物流导论', 'target': '高等数学BⅠ'}]

In [68]:
nodes

[{'category': 0, 'name': '工程制图', 'symbolSize': 68.9845},
 {'category': 0, 'name': '高等数学BⅠ', 'symbolSize': 35.5455},
 {'category': 0, 'name': '大学物理BI', 'symbolSize': 33.2915},
 {'category': 0, 'name': '测量学', 'symbolSize': 25.117},
 {'category': 0, 'name': '线性代数B', 'symbolSize': 31.34},
 {'category': 0, 'name': '军事技能训练', 'symbolSize': 34.4895},
 {'category': 0, 'name': '英语Ⅰ', 'symbolSize': 33.3985},
 {'category': 0, 'name': '高等数学BⅡ', 'symbolSize': 32.736000000000004},
 {'category': 0, 'name': '普通测量实验', 'symbolSize': 26.4615},
 {'category': 0, 'name': '综合运输概论', 'symbolSize': nan},
 {'category': 0, 'name': '物流导论', 'symbolSize': 31.174500000000002},
 {'category': 0, 'name': '分析化学', 'symbolSize': 25.927},
 {'category': 0, 'name': '物流学', 'symbolSize': 26.735},
 {'category': 0, 'name': '英语综合能力MⅠ', 'symbolSize': 27.677500000000002},
 {'category': 0, 'name': '计算机程序设计基础A', 'symbolSize': 26.4485},
 {'category': 0, 'name': '无机化学实验', 'symbolSize': nan},
 {'category': 0, 'name': '管理学原理', 'symbolSize'

In [73]:
def dump_obj(path,obj):
    '''先清空再保存对象到文件'''
    open(path,'w').close() # 先清空
    with open(path,'w') as f:
        json.dump(obj,f)

## 组合上述方法

In [72]:
def final_cal_relation(speciality_code,grade,store_links_path='links.txt',store_nodes_path='nodes.txt'):
    course_records = get_course_code(speciality_code,grade)
    print('已获取该专业课程')
    single_link = get_single_links(course_records)
    print('完成所有课程关系的计算')
    links,nodes = get_nodes_and_links(single_links)
    print('已生成nodes和links结构')
    # 存到文件里
    dump_obj(store_links_path,links)
    dump_obj(store_nodes_path,nodes)
    print('已存储在文件里')